#MATHGR5430 MACHINE LEARNING FOR FINANCE, SPRING 2025
<center>
<h1>Homework 3</h1>
</center>


## Background
You are a data scientist at a credit card company. A new dataset just landed in front of you. Your manager asked you to predict whether a client will make a large amount of payment(large 'payment' column). Before diving into modeling, you need to make the data usable. (Check datacard.txt) All the questions are based on this data set.

## Question 1

1.1 Data Cleaning

Import the dataset as a pandas dataframe. (Google is your best friend here, mount your google drive to colab and read your csv there)



Drop a irrelevant features (Hint: there is a column should never be use for any prediction and should not affect the prediction model, drop that one).



Identify and replace missing values (with any justifable approach to maintain datasize).



Identify and replace outliers. Justify your approach.



1.2 Exploratory Data Analysis

Plot the correlation matrix. What relationships or multicollinearity do you observe?



Provide summary statistics and visualizations (e.g., histograms, boxplots) for key variables like PURCHASES, CREDIT_LIMIT, PAYMENTS, MINIMUM_PAYMENTS, etc.



Create a binary label HIGH_PAYMENT where:
HIGH_PAYMENT = 1 if PAYMENTS > 4000 else 0

## Question 2 Supervised Learning with KNN and Ensemble Models Now that your dataset is cleaned and labeled, you will try to predict whether a customer will make a high payment using supervised learning.

2.1 Distance-Based Learning Normalize your features and explain why this step is important for KNN.



Train a K-Nearest Neighbors classifier to predict HIGH_PAYMENT.



Pick a reasonable k. (restrict to only 3-5 , you can try elbow method , it will not work well in this case), Justify your pick.



Evaluate your model using a confusion matrix and performance metrics: Accuracy, Precision, Recall, F1-score.



Try computing both Euclidean and Cosine distance manually for a small sample of data points.

2.2 Linear Discriminant Analysis (LDA)

Explain the concept of LDA and scatter matrices.



Apply LDA to reduce your data to justifyable dimensions.



Visualize the result and interpret the separation between classes.

2.3 Ensemble Learning Train two different classifiers:

Logistic Regression – adjust the regularization rates (try values like 0.01, 0.1, 1, 10).

SGDClassifier (Stochastic Gradient Descent Classifier) – experiment with different learning rates (eta0) and regularization penalties (l2, l1, etc.)

Suggested settings to try:

penalty = 'l2', eta0 = 0.01, 0.1, 1


Hint: If you are comfortable using third-party packages, you may optionally try XGBoostClassifier and explore learning_rate and reg_lambda.

Combine the two models using a simple ensemble method (e.g., majority voting).



Evaluate the performance of each model and the ensemble using:

Accuracy, Precision, Recall, F1-score



Briefly describe:

How changes in regularization and learning rate affected model performance.



Which combination performed best, and why you think that happened.

## Question 3 Unsupervised Learning and Dimensionality Reduction Your manager wants you to group customers based on their behavior and understand patterns without relying on labels.

3.1 Principal Component Analysis (PCA) Normalize the data.

Apply PCA and report the explained variance of the first 3 components.



Visualize the data in the PCA space (3D plot).



Interpret whether the data is well-represented in the new space.

3.2 K-Means Clustering Use the PCA-transformed data. Plot the Elbow Curve for k values from 2 to 10 using both random and kmeans++ initialization. find the optimal K.



Fit the final model and visualize the clusters in 3D.



Repeat clustering on the original (non-PCA) data. Choose 5 features and plot their distributions cluster by cluster.



Compare clustering results before and after PCA. Which works better and why?